In [ ]:
import json
from pathlib import Path
import shutil
import re
import subprocess
import sys

In [ ]:
spec_home = "/home/hchaudha/spec"

In [ ]:
def check_regex_in_file(file_path, regex_pattern):
    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Try to compile the regex pattern
    try:
        pattern = re.compile(regex_pattern)
    except re.error:
        print("Invalid regex pattern.")
        return False

    # Search for matches
    if pattern.search(content):
        return True
    else:
        return False

def replace_current_file(file_path, original_str, replaced_str):
    with open(file_path, 'r') as file:
        data = file.read()

    data, replaced_status = re.subn(original_str, replaced_str, data)
    if replaced_status != 0:
        print(f"""
Replaced in File: {file_path}
Original String: {original_str}
Replaced String: {replaced_str}
""")
    else:
        print(f"""
!!!!FAILED TO REPLACE!!!!
File path: {file_path}
Original String: {original_str}
Replaced String: {replaced_str}
""")
        sys.exit("Failed to replace parameters in files.")

    with open(file_path, 'w') as file:
        file.write(data)

def is_continuous(lst:list):
    # my_list = ["AZ", "BA", "BB", "BC"]
    # print(is_continuous(my_list))  # Should return True

    # my_list = ["AA", "AB", "AC", "AD"]
    # print(is_continuous(my_list))  # Should return True

    # my_list = ["AA", "AB", "AD", "AC"]
    # print(is_continuous(my_list))  # Should return False

    for i in range(len(lst) - 1):
        current = lst[i]
        next_elem = lst[i + 1]

        # Determine the next expected element
        if current[1] == 'Z':
            # If the second character is Z, the next expected should start the next sequence
            expected_next_first = chr(ord(current[0]) + 1)
            expected_next_second = 'A'
        else:
            # Otherwise, continue by incrementing the second character
            expected_next_first = current[0]
            expected_next_second = chr(ord(current[1]) + 1)

        expected_next_elem = expected_next_first + expected_next_second

        # If the next element is not the expected one, return False
        if next_elem != expected_next_elem:
            return False

    return True  # All elements are continuous

def verify_data_dict(data_dict:dict):
  for new_folder_name in data_dict.keys():
    data = data_dict[new_folder_name]
    if not data['new_run_path'].exists():
      raise Exception(f"""{data['new_run_path']=} does not exist!""")
    if not data['old_run_path'].exists():
      raise Exception(f"""{data['old_run_path']=} does not exist!""")
    
    levs_to_copy = data["levs_to_copy"]
    for lev_dict in levs_to_copy:
      folders_to_copy = lev_dict["folders_to_copy"]

      # Check that the segments are continuous and sorted
      if not is_continuous(folders_to_copy):
        raise Exception(f"{folders_to_copy=} is not a continuous list!")
  
      for folder in folders_to_copy:
        # Check that the levs we want to copy exist
        old_folder_path = data['old_run_path']/"Ev"/(lev_dict["old_lev"]+"_"+folder)
        if not old_folder_path.exists():
          raise Exception(f"{old_folder_path=} does not exist")

        # Check that the files being replace exist
        if folder == folders_to_copy[-1]:
          files_to_change_in_the_new_lev = lev_dict["files_to_change_in_the_new_lev"]
          for file_name in files_to_change_in_the_new_lev:
            file_path = old_folder_path/file_name
            if not file_path.exists():
              raise Exception(f"{file_path=} is supposed to be change in the new lev but it does not exist in the old folders lev.")

            # Check that the lists original_str and replaced_str have the same length
            original_str = files_to_change_in_the_new_lev[file_name]["original_str"]
            replaced_str = files_to_change_in_the_new_lev[file_name]["replaced_str"]
            if len(original_str) != len(replaced_str):
              raise Exception(f"{original_str=} and {replaced_str=} have different lengths!")

            # Check that the string that we are trying to replace actually exist in the first place
            for string_to_be_replaced in original_str:
              if not check_regex_in_file(file_path,string_to_be_replaced):
                raise Exception(f"{string_to_be_replaced=} not found in the file {file_path=}!")

def MakeNextSegment(EV_folder_path:Path, previous_segment_path:Path):
  command = f"cd {EV_folder_path} && {spec_home}/Support/bin/MakeNextSegment -d {previous_segment_path} -t . -S"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran MakeNextSegment in {EV_folder_path}: \n {status.stdout}")
  else:
    sys.exit(
        f"MakeNextSegment failed in {EV_folder_path} with error: \n {status.stdout} \n {status.stderr}")
    
def get_next_segment(lst):
    # my_list = ["AA", "AB", "AC", "AD", "BA"]
    # print(get_next_segment(my_list))  # Should return "BB"

    # my_list = ["AZ"]
    # print(get_next_segment(my_list))  # Should return "BA"

    # my_list = []
    # print(get_next_segment(my_list))  # Should return "AA"

    if not lst:
        return "AA"  # Return "AA" if the list is empty

    # Sort the list to ensure that elements are in the correct order
    lst.sort()

    # Retrieve the last element from the sorted list
    last_elem = lst[-1]

    # Determine the next element
    if last_elem[1] == 'Z':
        # If the second character is 'Z', increment the first character and set the second to 'A'
        next_first = chr(ord(last_elem[0]) + 1)
        next_second = 'A'
    else:
        # Otherwise, simply increment the second character
        next_first = last_elem[0]
        next_second = chr(ord(last_elem[1]) + 1)

    next_elem = next_first + next_second
    return next_elem

In [ ]:
# Ev is always made, the idea is to make things simple and consistent
data_dict={
  "Lev3_ah100":{
    "spec_home": Path("/home/hchaudha/spec"),
    "new_run_path":Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L34_variations"),
    "old_run_path":Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master"),
    "link_or_copy_folders":"link",
    "copy_ID":False,
    "copy_bin":True,
    "levs_to_copy":[
      {
        "old_lev":"Lev3",
        "new_name":"Lev3",
        "folders_to_copy":["AA","AB"],
        "files_to_change_in_the_new_lev":{
          "AmrDriver.input":{
          "original_str": [
            "InsaneResolution    = 400;"
          ],
          "replaced_str": [
            "Evolve = 0;"
          ]
          },
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;"
          ],
          "replaced_str": [
            "FinalTime = 2500;"
          ]
          }
        },
      },
      {
        "old_lev":"Lev3",
        "new_name":"Lev5",
        "folders_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;"
          ],
          "replaced_str": [
            "FinalTime = 2500;"
          ]
          }
        },
      }
    ],
  }
}

In [ ]:
def verify_data_dict(data_dict:dict):
  for new_folder_name in data_dict.keys():
    data = data_dict[new_folder_name]
    if not data['new_run_path'].exists():
      raise Exception(f"""{data['new_run_path']=} does not exist!""")
    if not data['old_run_path'].exists():
      raise Exception(f"""{data['old_run_path']=} does not exist!""")
    
    if data["copy_ID"]:
      old_ID_path = data['old_run_path']/"ID"
      if not old_ID_path.exists():
        raise Exception(f"ID folder marked for copy but {old_ID_path=} does not exist.")    
    
    if data["copy_bin"]:
      old_bin_path = data['old_run_path']/"bin"
      if not old_bin_path.exists():
        raise Exception(f"bin folder marked for copy but {old_bin_path=} does not exist.")    
      
    levs_to_copy = data["levs_to_copy"]
    for lev_dict in levs_to_copy:
      folders_to_copy = lev_dict["folders_to_copy"]

      # Check that the segments are continuous and sorted
      if not is_continuous(folders_to_copy):
        raise Exception(f"{folders_to_copy=} is not a continuous list!")
  
      for folder in folders_to_copy:
        # Check that the levs we want to copy exist
        old_folder_path = data['old_run_path']/"Ev"/(lev_dict["old_lev"]+"_"+folder)
        if not old_folder_path.exists():
          raise Exception(f"{old_folder_path=} does not exist")

        # Check that the files being replace exist
        if folder == folders_to_copy[-1]:
          files_to_change_in_the_new_lev = lev_dict["files_to_change_in_the_new_lev"]
          for file_name in files_to_change_in_the_new_lev:
            file_path = old_folder_path/file_name
            if not file_path.exists():
              raise Exception(f"{file_path=} is supposed to be change in the new lev but it does not exist in the old folders lev.")

            # Check that the lists original_str and replaced_str have the same length
            original_str = files_to_change_in_the_new_lev[file_name]["original_str"]
            replaced_str = files_to_change_in_the_new_lev[file_name]["replaced_str"]
            if len(original_str) != len(replaced_str):
              raise Exception(f"{original_str=} and {replaced_str=} have different lengths!")
            for string_to_be_replaced in original_str:
              if not check_regex_in_file(file_path,string_to_be_replaced):
                raise Exception(f"{string_to_be_replaced=} not found in the file {file_path=}!")



In [ ]:
verify_data_dict(data_dict)

In [ ]:


def link_or_copy(source_path: Path, destination_path: Path, link_or_copy_folders: str):
    if link_or_copy_folders == "link":
        # Create a symbolic link
        if not destination_path.exists():
            destination_path.symlink_to(source_path)
            print(f"Created a symbolic link from {source_path} to {destination_path}")
        else:
            print(f"Destination {destination_path} already exists. Cannot create a link.")
    
    elif link_or_copy_folders == "copy":
        # Copy the directory using shutil when using pathlib
        if not destination_path.exists():
            shutil.copytree(source_path, destination_path)
            print(f"Copied {source_path} to {destination_path}")
        else:
            print(f"Destination {destination_path} already exists. Cannot copy.")
    
    else:
        print("Invalid option. Please use 'link' or 'copy'.")


def create_new_segment(folder_name:str,data_dict:dict):
  folder_dict = data_dict[folder_name]
  new_run_path:Path = folder_dict["new_run_path"]
  new_segment_path = new_run_path/folder_name
  old_run_path:Path = folder_dict["old_run_path"]
  link_or_copy_folders = folder_dict["link_or_copy_folders"]
  copy_ID = folder_dict["copy_ID"]
  old_ID_path = old_run_path/"ID"
  copy_bin = folder_dict["copy_bin"]
  old_bin_path = old_run_path/"bin"

  levs_to_copy = folder_dict["levs_to_copy"]
  
  new_segment_path.mkdir()
  new_Ev_path = new_run_path/"Ev"
  new_ID_path = new_run_path/"ID"
  new_bin_path = new_run_path/"Ev/bin"
  new_Ev_path.mkdir()
  if copy_ID:
    link_or_copy(old_ID_path,new_ID_path,link_or_copy_folders)
  if copy_bin:
    link_or_copy(old_bin_path,new_bin_path,link_or_copy_folders)

  levs_to_copy = folder_dict["levs_to_copy"]
  for lev_dict in levs_to_copy:
    folders_to_copy = lev_dict["folders_to_copy"]

    for folder in folders_to_copy:
      # Check that the levs we want to copy exist
      old_folder_path = folder_dict['old_run_path']/"Ev"/(lev_dict["old_lev"]+"_"+folder)
      new_folder_path = folder_dict['new_run_path']/"Ev"/(lev_dict["new_name"]+"_"+folder)
      link_or_copy(old_folder_path,new_folder_path,link_or_copy_folders)

      # Make the next segment
      new_segment_path
    
    if not old_folder_path.exists():
      raise Exception(f"{old_folder_path=} does not exist")

    # Check that the files being replace exist
    if folder == folders_to_copy[-1]:
      files_to_change_in_the_new_lev = lev_dict["files_to_change_in_the_new_lev"]
      for file_name in files_to_change_in_the_new_lev:
        file_path = old_folder_path/file_name
        if not file_path.exists():
          raise Exception(f"{file_path=} is supposed to be change in the new lev but it does not exist in the old folders lev.")

        # Check that the lists original_str and replaced_str have the same length
        original_str = files_to_change_in_the_new_lev[file_name]["original_str"]
        replaced_str = files_to_change_in_the_new_lev[file_name]["replaced_str"]
        if len(original_str) != len(replaced_str):
          raise Exception(f"{original_str=} and {replaced_str=} have different lengths!")
        for string_to_be_replaced in original_str:
          if not check_regex_in_file(file_path,string_to_be_replaced):
            raise Exception(f"{string_to_be_replaced=} not found in the file {file_path=}!")
    

  

    


  

In [ ]:
link_or_copy(
  Path("/central/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra_200/Lev5_AJ"),
  Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L34_variations/Ev/Lev516"),
  "copy"
)